In [36]:
#IMPORTATION DE MODULES
#from textblob import TextBlob
import json
import re
import string
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from nltk.corpus import stopwords



In [138]:
# Télécharger les stopwords en français si ce n'est pas déjà fait
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')



[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/lnberroug/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/lnberroug/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /Users/lnberroug/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [15]:
#PARTIE : CREATION DU TWEET + SOUMISSION DU TWEET DANS LANDING ZONE

#Création d'une Classe Tweet à laquelle seront associées les attributs suivant : auteurs, hashtag, mentions, sentiments, (topics???)

class Tweet: #Plan/Modèle
    def __init__(self, auteur, contenu): #Méthode (assisstant qui aide à donner à chaque tweet ses attributs)
        self.auteur = auteur
        self.contenu = contenu
        self.hashtags = self.Extraction_Hashtags()
        self.mentions = self.Extraction_Mentions()
        self.sentiment = self.Extraction_Sentiments()

    def Nettoyage_Tweet(self):
        """Fonction qui supprime les caractères spéciaux d'un tweet"""
        enlever_speciaux = re.sub(r'[^a-zA-Z0-9\s@#]', '', self.contenu)
        return enlever_speciaux
    
    def Zone_Atterisssage (Fichier): # incomplet
        """Stockage du tweet dans le fichier json zone d'atterissage"""
        
        ZA=[]
        with open (Fichier,'r',encoding='utf-8') as f:
            donnee= json.load(f)
            
        for infos_tweets in donnee:
            auteur = infos_tweets.get("author_id")
            contenu = infos_tweets.get("text")
            tweet_auteur = {
                "id": auteur,
                "tweet": Nettoyage_Tweet(contenu)
            }
            ZA.append(tweet_auteur)
        
        with open('ZA.json','w', encoding='utf-8') as f:
            json.dump(ZA, f, indent=4, ensure_ascii=False)
    

    
    def Extraction_Sentiments(self): 
        """Extrait le sentiment du texte"""
        blob = TextBlob(self.contenu)
        polarite =  blob.sentiment.polarity
        if polarite > 0:
            sentiment = 'positif'
        elif polarite < 0:
            sentiment = 'négatif'
        elif polarite == 0:
            sentiment = 'neutre'
        return sentiment
    
    
    def Extraction_Hashtags():  #A Faire 
        """"""
        hashtags = re.findall(r'#\w+', tweet)
        return hashtags
    
    def Extraction_Mentions(self):  #A Faire
        """Fonction qui extrait les mentions de personnes dans un tweet"""
        mentions = re.findall(r'@[A-Za-z0-9_]+', self.contenu)

        return mentions


    def Id_Topic(tweet):

        def traitement_texte(text):
            text= text.lower() # mettre en minuscule
            text= re.sub(r'[^a-z\s]', '', text) 
            return text
        tweets= []
        with open ('ZA.json','r',encoding='utf-8') as f:
            donnee= json.load(f)
        
        tweets = [infos["tweet"] for infos in donnee]
        
        texte_traite = [traitement_texte(tweet) for tweet in tweets]
        
        vecteur = TfidfVectorizer(stop_words= stopwords.words('french'))  # Utilise les stopwords en français intégrés dans sklearn
        X = vecteur.fit_transform(texte_traite)

        lda = LatentDirichletAllocation(n_components=2, random_state=42)
        lda.fit(X)

        topics = []
        words = vecteur.get_feature_names_out()
        for i, topic_weights in enumerate(lda.components_):
            top_indices = topic_weights.argsort()[:-3:-1]  # Prendre les 2 mots les plus importants
            top_words = [words[i] for i in top_indices]
            topics.append(f": {', '.join(top_words)}")


        tweet_traite = traitement_texte(tweet)
        vecteur = TfidfVectorizer(stop_words='french')  # Utilise les stopwords en français intégrés dans sklearn
        vecteur_tweet= vecteur.transform([tweet_traite])
        probabilities = lda.transform(vecteur_tweet)
        topic = probabilities.argmax()  # Trouve l'indice du topic avec la probabilité la plus élevée

        print(f"Le tweet :\n{tweet}\nAppartient au Topic {topics[topic]}")




 
#tweet = Tweet (auteur, contenu) # Utilisation de la class Tweet
        

# Process chaque tweet:
                
with open ('ZA.json','r') as file:
    tweets = json.load(file)


for tweet in tweets:
    print(f"Auteur : {tweet.auteur}")
    print(f"Contenu : {tweet.contenu}")
    print(f"Hashtags : {tweet.hashtags}")
    print(f"Mentions : {tweet.mentions}")
    print(f"Sentiment : {tweet.sentiment}")

# --> il faut charger ces données dans une date frame : df = pd.DataFrame(tweets)

# yo




SyntaxError: invalid syntax (515494132.py, line 26)

Trucs à revoir

On utilise la class avec le json Versailles_tweets alors que tous les traitements sont fait à partir de la zone d'atterissage. Il faut changer ça, peut être créer une autre class ? ou juste utilser le tweet.

In [148]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()


ZA = [
    {"tweet": "Le gouvernement a annoncé un plan pour soutenir les athlètes locaux en vue des prochains Jeux Olympiques."},
    {"tweet": "Les débats sur la politique fiscale affectent directement le financement des clubs sportifs nationaux."},
    {"tweet": "Les politiques sportives du gouvernement sont critiquées par certains partis de l'opposition."},
    {"tweet": "Les joueurs de football et les politiques de sécurité sont au centre des discussions cette semaine."},
    {"tweet": "Le ministre des Sports a proposé une nouvelle loi pour améliorer les infrastructures sportives en France."},
    {"tweet": "L'impact des réformes économiques pourrait affecter la préparation des athlètes pour les compétitions internationales."},
    {"tweet": "Les équipes de football sont en plein préparatifs pour la Coupe du Monde, malgré les réformes sociales en cours."},
    {"tweet": "Les débats autour des droits des travailleurs sportifs sont au cœur des discussions législatives."},
    {"tweet": "Le président a annoncé une série de mesures pour encourager la pratique du sport à tous les niveaux."},
    {"tweet": "Les discussions sur la sécurité nationale influencent les événements sportifs internationaux, comme la Coupe du Monde."},
    {"tweet": "Les athlètes se battent pour leurs droits, tout comme les députés débattent des nouvelles réformes sociales."},
    {"tweet": "La politique de financement des événements sportifs nationaux est une priorité pour le gouvernement."},
    {"tweet": "Les partisans du football et de la réforme politique se sont réunis pour discuter des changements attendus."},
    {"tweet": "Le gouvernement tente de promouvoir les sports tout en faisant face à des crises économiques et sociales."},
    {"tweet": "Les réformes fiscales proposées par le gouvernement pourraient affecter les subventions accordées aux clubs de football."},
    {"tweet": "Les athlètes sont appelés à soutenir la réforme du système de santé qui impacte leur bien-être."},
    {"tweet": "Le match de football est plus qu'un simple jeu, c'est aussi un moyen pour le gouvernement de promouvoir ses politiques."},
    {"tweet": "Les leaders politiques de différents pays se sont réunis pour discuter de la gestion des événements sportifs internationaux."},
    {"tweet": "Les athlètes soutiennent de plus en plus les initiatives politiques visant à améliorer les conditions de vie des jeunes."},
    {"tweet": "Les réformes politiques en matière d'urbanisme influencent la construction de nouveaux stades pour les compétitions sportives."},
    {"tweet": "La diplomatie sportive est devenue un outil pour apaiser les tensions politiques entre les nations."},
    {"tweet": "Les athlètes de haut niveau se mobilisent pour des changements dans la politique de santé publique."},
    {"tweet": "La politique de droits de l'homme influence le choix des pays hôtes pour les événements sportifs mondiaux."},
    {"tweet": "La gestion des crises politiques peut avoir un impact direct sur l'organisation des événements sportifs internationaux."},
    {"tweet": "Les sportifs ont exprimé leur soutien à la réforme de la loi sur l'immigration, ce qui a créé un débat politique."}
]

def Id_Topic(tweet):

        def traitement_texte(text):
            text= text.lower() # mettre en minuscule
            text= re.sub(r'[^a-z\s]', '', text)
            text = " ".join([lemmatizer.lemmatize(word) for word in text.split()])
            return text
        def get_stopwords():
            return stopwords.words('french')
        stopwords_list = get_stopwords()
        tweets= []
        with open ('ZA.json','r',encoding='utf-8') as f:
            donnee= json.load(f)
        
        tweets = [infos["tweet"] for infos in donnee]
        
        texte_traite = [traitement_texte(tweet) for tweet in tweets]
        
        vecteur = TfidfVectorizer(stop_words= stopwords_list)  # Utilise les stopwords en français intégrés dans sklearn
        X = vecteur.fit_transform(texte_traite)

        lda = LatentDirichletAllocation(n_components=3, random_state=100)
        lda.fit(X)


        topics = []
        words = vecteur.get_feature_names_out()
        for i, topic_weights in enumerate(lda.components_):
            top_indices = topic_weights.argsort()[:-2:-1]  # Prendre les 2 mots les plus importants
            top_words = [words[i] for i in top_indices]
            topics.append(f": {', '.join(top_words)}")


        tweet_traite = traitement_texte(tweet)
        vecteur_tweet= vecteur.transform([tweet_traite])
        probabilities = lda.transform(vecteur_tweet)
        topic = probabilities.argmax()  # Trouve l'indice du topic avec la probabilité la plus élevée

        for to in topics:
            print (to)

        print(f"Le tweet :{tweet}\nAppartient au Topic {topics[topic]}")


tweet = "Le ministre des Sports a proposé une nouvelle loi pour améliorer les infrastructures sportives en France."
Id_Topic(tweet)


: fte
: entreprises
: annonc
Le tweet :Le ministre des Sports a proposé une nouvelle loi pour améliorer les infrastructures sportives en France.
Appartient au Topic : entreprises


--> Pour améliorer ce code il faudrait utiliser nltk pour la tokenisation et la lemmenisation : ça va couper les tweet en petit mots et reduire les mots en leur forme conique ( je n'arrive pas à utilser nltk pour faire ça, erreur)

Ducoup la c pas précis